# Démonstration : Onion Routing & Tor

**Objectifs** :
- Comprendre le principe du routage en oignon
- Simuler le protocole Tor
- Démontrer les garanties de confidentialité
- Analyser les limites et attaques

In [ ]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
import secrets
import hashlib

## 1. Onion Routing : Principe

**Problème** : Alice veut communiquer avec un serveur sans révéler son identité.

**Solution** : Routage en oignon (Onion Routing, Chaum 1981)

**Principe** :
1. Alice choisit un **circuit** de relais : $R_1, R_2, R_3$
2. Alice chiffre le message en couches (comme un oignon) :
   - $m_3 = E_{k_3}(\text{destination} \| \text{message})$
   - $m_2 = E_{k_2}(R_3 \| m_3)$
   - $m_1 = E_{k_1}(R_2 \| m_2)$
3. Alice envoie $m_1$ à $R_1$
4. $R_1$ déchiffre, voit $R_2$, transmet $m_2$ à $R_2$
5. $R_2$ déchiffre, voit $R_3$, transmet $m_3$ à $R_3$
6. $R_3$ déchiffre, voit la destination, transmet le message

**Propriétés** :
- ✅ $R_1$ connaît Alice mais pas la destination
- ✅ $R_3$ connaît la destination mais pas Alice
- ✅ $R_2$ ne connaît ni Alice ni la destination
- ✅ Observateur externe ne voit que du trafic chiffré

## 2. Simulation Simplifiée d'Onion Routing

In [ ]:
class OnionRelay:
    """
    Relais d'un circuit onion routing.
    """
    def __init__(self, name: str):
        self.name = name
        # Générer paire de clés pour échange de clés
        self.private_key = x25519.X25519PrivateKey.generate()
        self.public_key = self.private_key.public_key()
        print(f"🔐 {name} initialisé avec clé publique")
    
    def establish_key(self, client_public_key: x25519.X25519PublicKey) -> bytes:
        """
        Établit une clé symétrique avec le client (DH).
        
        Returns:
            Clé symétrique dérivée
        """
        # Diffie-Hellman
        shared_secret = self.private_key.exchange(client_public_key)
        
        # Dérivation de clé (HKDF)
        kdf = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b'onion routing key',
            backend=default_backend()
        )
        key = kdf.derive(shared_secret)
        
        return key
    
    def peel_layer(self, onion: bytes, key: bytes) -> tuple:
        """
        Déchiffre une couche de l'oignon.
        
        Args:
            onion: Données chiffrées
            key: Clé symétrique
        
        Returns:
            (next_hop, remaining_onion)
        """
        # Séparer nonce et ciphertext
        nonce = onion[:12]
        ciphertext = onion[12:]
        
        # Déchiffrer
        aesgcm = AESGCM(key)
        plaintext = aesgcm.decrypt(nonce, ciphertext, None)
        
        # Format : next_hop_length (1 byte) | next_hop | payload
        next_hop_length = plaintext[0]
        next_hop = plaintext[1:1+next_hop_length].decode('utf-8')
        payload = plaintext[1+next_hop_length:]
        
        print(f"   {self.name} : Couche déchiffrée, prochaine étape → {next_hop}")
        
        return next_hop, payload

def onion_routing_demo():
    """
    Simulation d'onion routing avec 3 relais.
    """
    print("=" * 70)
    print("ONION ROUTING : Simulation")
    print("=" * 70)
    
    # Créer 3 relais
    print("\n🌐 Création du réseau (3 relais) :")
    relay1 = OnionRelay("Relay 1 (Entry/Guard)")
    relay2 = OnionRelay("Relay 2 (Middle)")
    relay3 = OnionRelay("Relay 3 (Exit)")
    
    # Alice établit des clés avec chaque relais
    print("\n👩 Alice établit des clés avec chaque relais (DH) :")
    
    # Clé avec Relay 1
    alice_private_1 = x25519.X25519PrivateKey.generate()
    alice_public_1 = alice_private_1.public_key()
    key1 = relay1.establish_key(alice_public_1)
    # Alice calcule aussi
    shared1 = alice_private_1.exchange(relay1.public_key)
    kdf1 = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'onion routing key', backend=default_backend())
    alice_key1 = kdf1.derive(shared1)
    
    # Clé avec Relay 2
    alice_private_2 = x25519.X25519PrivateKey.generate()
    alice_public_2 = alice_private_2.public_key()
    key2 = relay2.establish_key(alice_public_2)
    shared2 = alice_private_2.exchange(relay2.public_key)
    kdf2 = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'onion routing key', backend=default_backend())
    alice_key2 = kdf2.derive(shared2)
    
    # Clé avec Relay 3
    alice_private_3 = x25519.X25519PrivateKey.generate()
    alice_public_3 = alice_private_3.public_key()
    key3 = relay3.establish_key(alice_public_3)
    shared3 = alice_private_3.exchange(relay3.public_key)
    kdf3 = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'onion routing key', backend=default_backend())
    alice_key3 = kdf3.derive(shared3)
    
    print("   ✅ Clés établies avec les 3 relais")
    
    # Message à envoyer
    destination = "example.com"
    message = b"GET / HTTP/1.1\r\nHost: example.com\r\n\r\n"
    
    print(f"\n📝 Alice veut envoyer :")
    print(f"   Destination : {destination}")
    print(f"   Message : {message[:50]}...")
    
    # Construction de l'oignon (de l'intérieur vers l'extérieur)
    print(f"\n🧅 Construction de l'oignon (3 couches) :")
    
    # Couche 3 (la plus interne) : destination + message
    dest_bytes = destination.encode('utf-8')
    layer3_plaintext = bytes([len(dest_bytes)]) + dest_bytes + message
    nonce3 = secrets.token_bytes(12)
    aesgcm3 = AESGCM(alice_key3)
    layer3_ciphertext = aesgcm3.encrypt(nonce3, layer3_plaintext, None)
    layer3 = nonce3 + layer3_ciphertext
    
    print(f"   Couche 3 : Chiffré avec clé Relay3, destination = {destination}")
    
    # Couche 2 : Relay 3 + layer3
    relay3_name = relay3.name.encode('utf-8')
    layer2_plaintext = bytes([len(relay3_name)]) + relay3_name + layer3
    nonce2 = secrets.token_bytes(12)
    aesgcm2 = AESGCM(alice_key2)
    layer2_ciphertext = aesgcm2.encrypt(nonce2, layer2_plaintext, None)
    layer2 = nonce2 + layer2_ciphertext
    
    print(f"   Couche 2 : Chiffré avec clé Relay2, next hop = Relay 3")
    
    # Couche 1 (extérieure) : Relay 2 + layer2
    relay2_name = relay2.name.encode('utf-8')
    layer1_plaintext = bytes([len(relay2_name)]) + relay2_name + layer2
    nonce1 = secrets.token_bytes(12)
    aesgcm1 = AESGCM(alice_key1)
    layer1_ciphertext = aesgcm1.encrypt(nonce1, layer1_plaintext, None)
    onion = nonce1 + layer1_ciphertext
    
    print(f"   Couche 1 : Chiffré avec clé Relay1, next hop = Relay 2")
    print(f"\n   Oignon construit : {len(onion)} bytes")
    
    # Transmission à travers le circuit
    print(f"\n📡 Transmission à travers le circuit :")
    
    # Relay 1 (Entry/Guard)
    print(f"\n1️⃣ Alice → Relay 1")
    next_hop1, payload1 = relay1.peel_layer(onion, alice_key1)
    print(f"   Relay 1 sait : Provenance=Alice, Destination=?")
    
    # Relay 2 (Middle)
    print(f"\n2️⃣ Relay 1 → Relay 2")
    next_hop2, payload2 = relay2.peel_layer(payload1, alice_key2)
    print(f"   Relay 2 sait : Provenance=?, Destination=?")
    
    # Relay 3 (Exit)
    print(f"\n3️⃣ Relay 2 → Relay 3")
    destination_final, message_final = relay3.peel_layer(payload2, alice_key3)
    print(f"   Relay 3 sait : Provenance=?, Destination={destination_final}")
    
    print(f"\n4️⃣ Relay 3 → {destination_final}")
    print(f"   Message délivré : {message_final[:50]}...")
    
    # Garanties de confidentialité
    print(f"\n" + "=" * 70)
    print(f"GARANTIES DE CONFIDENTIALITÉ")
    print(f"=" * 70)
    
    print(f"\n✅ Relay 1 (Entry/Guard) connaît :")
    print(f"   - Identité d'Alice (adresse IP)")
    print(f"   - Relay 2 (prochain saut)")
    print(f"   - ❌ PAS la destination finale")
    print(f"   - ❌ PAS le contenu du message")
    
    print(f"\n✅ Relay 2 (Middle) connaît :")
    print(f"   - Relay 1 (saut précédent)")
    print(f"   - Relay 3 (prochain saut)")
    print(f"   - ❌ PAS l'identité d'Alice")
    print(f"   - ❌ PAS la destination finale")
    print(f"   - ❌ PAS le contenu du message")
    
    print(f"\n✅ Relay 3 (Exit) connaît :")
    print(f"   - Relay 2 (saut précédent)")
    print(f"   - Destination finale ({destination_final})")
    print(f"   - Contenu du message (si pas chiffré end-to-end)")
    print(f"   - ❌ PAS l'identité d'Alice")
    
    print(f"\n💡 Propriété clé : Aucun relais ne connaît à la fois")
    print(f"   l'émetteur ET le destinataire !")

onion_routing_demo()

## 3. Tor : The Onion Router

**Historique** :
- 1995 : Onion Routing (US Naval Research Lab)
- 2002 : Tor (The Onion Router) développé
- 2006 : Tor Project fondé (organisation à but non lucratif)
- Aujourd'hui : ~2 millions d'utilisateurs quotidiens

**Architecture** :
- **Guard Node** (Entry) : Connaît l'utilisateur
- **Middle Node** : Ne connaît ni émetteur ni destination
- **Exit Node** : Connaît la destination
- **Circuit** : 3 relais (compromis sécurité/performance)
- **Rotation** : Nouveau circuit toutes les 10 minutes

**Protocoles** :
- **Établissement de circuit** : Diffie-Hellman incrémental
- **Chiffrement** : AES-CTR (3 couches)
- **Cellules** : Messages de 512 bytes
- **Hidden Services** : Adresses .onion

In [ ]:
print("=" * 70)
print("TOR : Architecture et Statistiques")
print("=" * 70)

print("\n📊 Réseau Tor (estimations 2025) :")
print(f"   Relais actifs : ~7000-8000")
print(f"   Exit nodes : ~1500-2000 (~25%)")
print(f"   Bande passante totale : ~300-400 Gbps")
print(f"   Utilisateurs quotidiens : ~2-3 millions")

print("\n🌍 Distribution géographique (top 5) :")
print(f"   1. Allemagne : ~20%")
print(f"   2. États-Unis : ~15%")
print(f"   3. France : ~10%")
print(f"   4. Pays-Bas : ~8%")
print(f"   5. Royaume-Uni : ~6%")

print("\n🔐 Cryptographie utilisée :")
print(f"   Échange de clés : X25519 (ECDH Curve25519)")
print(f"   Authentification relais : Ed25519")
print(f"   Chiffrement couches : AES-256-CTR")
print(f"   Intégrité : SHA-256 (HMAC)")
print(f"   Hidden services v3 : Ed25519 + X25519")

print("\n⏱️  Performance :")
print(f"   Latence ajoutée : ~100-300 ms (vs connexion directe)")
print(f"   Vitesse : ~1-5 Mbps (dépend du circuit)")
print(f"   Overhead : 3 couches de chiffrement")

print("\n🎯 Cas d'usage :")
print(f"   ✅ Navigation anonyme (dissidents, journalistes)")
print(f"   ✅ Contournement de censure (Chine, Iran, etc.)")
print(f"   ✅ Protection contre surveillance de masse")
print(f"   ✅ Hidden services (.onion) : Wikileaks, ProtonMail, etc.")
print(f"   ⚠️  Aussi utilisé pour activités illégales (darknet markets)")

## 4. Limitations et Attaques

**Tor n'est PAS une solution magique !**

In [ ]:
print("\n" + "=" * 70)
print("LIMITATIONS ET ATTAQUES SUR TOR")
print("=" * 70)

print("\n❌ 1. Traffic Analysis (Corrélation de trafic)")
print(f"   Scénario : Adversaire contrôle Entry ET Exit nodes")
print(f"   Attaque : Corrélation temporelle (timing, volume)")
print(f"   Résultat : Peut lier utilisateur et destination")
print(f"   Protection : Probabilité faible si réseau large")

print("\n❌ 2. Exit Node Eavesdropping")
print(f"   Scénario : Exit node malveillant")
print(f"   Attaque : Lit trafic non chiffré (HTTP, FTP)")
print(f"   Résultat : Peut voir mots de passe, cookies, etc.")
print(f"   Protection : Toujours utiliser HTTPS (end-to-end encryption)")

print("\n❌ 3. Website Fingerprinting")
print(f"   Scénario : Adversaire observe trafic chiffré (Entry)")
print(f"   Attaque : Analyse taille/timing des paquets")
print(f"   Résultat : Peut deviner site visité (~90% précision)")
print(f"   Protection : Padding, traffic shaping (partiellement dans Tor)")

print("\n❌ 4. Sybil Attack (Contrôle de nombreux relais)")
print(f"   Scénario : Adversaire lance de nombreux relais malveillants")
print(f"   Attaque : Augmente probabilité d'être dans le circuit")
print(f"   Résultat : Traffic analysis, dé-anonymisation")
print(f"   Protection : Tor limite autorité relais, détection anomalies")

print("\n❌ 5. Browser Exploits / Application Leaks")
print(f"   Scénario : JavaScript, WebRTC, plugins révèlent IP réelle")
print(f"   Attaque : Exploits ciblés (FBI Playpen 2015)")
print(f"   Résultat : Dé-anonymisation complète")
print(f"   Protection : Tor Browser (JavaScript limité, pas de plugins)")

print("\n❌ 6. Global Passive Adversary")
print(f"   Scénario : NSA/GCHQ observe TOUT le trafic Internet")
print(f"   Attaque : Corrélation globale (timing, volume)")
print(f"   Résultat : Peut potentiellement dé-anonymiser")
print(f"   Protection : Aucune protection complète (hors scope de Tor)")

print("\n❌ 7. Hidden Service Localization")
print(f"   Scénario : Localiser serveur .onion")
print(f"   Attaque : Traffic analysis, timing attacks")
print(f"   Résultat : Peut révéler localisation du serveur")
print(f"   Protection : Guard nodes, vanguards (Tor v3)")

print("\n✅ BONNES PRATIQUES :")
print(f"   1. Toujours utiliser HTTPS (pas seulement Tor)")
print(f"   2. Utiliser Tor Browser (pas juste le réseau Tor)")
print(f"   3. Ne pas se connecter à comptes personnels via Tor")
print(f"   4. Désactiver JavaScript sur sites non fiables")
print(f"   5. Ne pas télécharger/ouvrir fichiers via Tor")
print(f"   6. Combiner avec VPN (débat : avant ou après Tor ?)")

## 5. Hidden Services (.onion)

**Principe** : Serveurs accessibles uniquement via Tor, sans révéler leur localisation.

**Fonctionnement** :
1. Serveur génère paire de clés Ed25519
2. Adresse .onion = Clé publique (v3: 56 caractères)
3. Serveur crée circuits vers points de rendez-vous
4. Client crée circuit vers point de rendez-vous
5. Communication via point de rendez-vous (6 sauts total !)

**Avantages** :
- ✅ Serveur anonyme (localisation cachée)
- ✅ Résistant à la censure (pas de DNS, pas d'IP)
- ✅ Authentification (adresse = clé publique)

**Inconvénients** :
- ⚠️ Plus lent (6 sauts vs 3)
- ⚠️ Complexité accrue

In [ ]:
print("\n" + "=" * 70)
print("HIDDEN SERVICES (.onion)")
print("=" * 70)

print("\n🌐 Exemples de sites .onion légitimes :")
print(f"   ProtonMail : protonmailrmez3lotccipshtkleegetolb73fuirgj7r4o4vfu7ozyd.onion")
print(f"   DuckDuckGo : duckduckgogg42xjoc72x3sjasowoarfbgcmvfimaftt6twagswzczad.onion")
print(f"   NYTimes : nytimesn7cgmftshazwhfgzm37qxb44r64ytbb2dj3x62d2lljsciiyd.onion")
print(f"   BBC News : bbcnewsd73hkzno2ini43t4gblxvycyac5aw4gnv7t2rccijh7745uqd.onion")

print("\n🔐 Génération d'adresse .onion v3 :")
print(f"   1. Générer paire Ed25519")
print(f"   2. Clé publique : 32 bytes")
print(f"   3. Adresse = base32(pubkey | checksum | version) + '.onion'")
print(f"   4. Longueur : 56 caractères")

# Simulation de génération d'adresse
from cryptography.hazmat.primitives.asymmetric import ed25519
import base64

# Générer clé
private_key = ed25519.Ed25519PrivateKey.generate()
public_key = private_key.public_key()

# Extraire bytes de la clé publique
public_bytes = public_key.public_bytes(
    encoding=serialization.Encoding.Raw,
    format=serialization.PublicFormat.Raw
)

# Calculer checksum simplifié (vraie formule plus complexe)
checksum = hashlib.sha256(b'.onion checksum' + public_bytes + bytes([3])).digest()[:2]

# Adresse onion (simplifié)
onion_address_bytes = public_bytes + checksum + bytes([3])  # version 3
onion_address = base64.b32encode(onion_address_bytes).decode('utf-8').lower().rstrip('=') + '.onion'

print(f"\n🆔 Adresse .onion générée (exemple) :")
print(f"   {onion_address}")
print(f"   ✅ Auto-authentifiée (adresse = hash de clé publique)")

print("\n📊 Circuit Hidden Service :")
print(f"   Client → Guard → Middle → Rendez-vous")
print(f"   Serveur → Guard → Middle → Rendez-vous")
print(f"   Total : 6 sauts (3 + 3)")
print(f"   Latence : ~200-600 ms (vs ~100-300 ms pour Tor normal)")

## Conclusion

**Points clés** :
- Onion Routing : Chiffrement multi-couches pour anonymat
- ✅ Tor : 3 relais (Entry, Middle, Exit), circuits rotatifs
- ✅ Aucun relais ne connaît à la fois émetteur et destinataire
- ✅ Hidden Services : Serveurs anonymes (.onion)
- ⚠️ Limitations : Traffic analysis, exit node attacks, fingerprinting

**Sécurité** :
- Protection contre surveillance locale/ISP : ✅ Excellente
- Protection contre adversaire global : ⚠️ Limitée
- Protection contre exit node malveillant : ❌ Nécessite HTTPS

**En pratique** :
- Tor Browser : Meilleure option (JavaScript limité, pas de plugins)
- Toujours utiliser HTTPS avec Tor
- Ne pas mélanger trafic anonyme et personnel
- Comprendre les limites (pas une solution magique)

**Alternatives** :
- I2P : Réseau anonyme peer-to-peer
- Freenet : Stockage distribué anonyme
- VPN : Protection simple mais moins anonyme (VPN connaît identité)